In [1]:
import sqlite3
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

In [2]:
def load_data():
    conn = sqlite3.connect("../Data/yelpResData.db")
    conn.text_factory = lambda x: str(x, 'gb2312', 'ignore')
    cursor = conn.cursor()
    
    # Create Review DataFrame
    cursor.execute(
        "SELECT reviewID, reviewerID, restaurantID, date, rating, usefulCount as reviewUsefulCount, reviewContent, flagged FROM review WHERE flagged in ('Y','N')")
    review_df = pd.DataFrame(cursor.fetchall(), columns=[column[0] for column in cursor.description])

    # Create Reviewer DataFrame
    cursor.execute("SELECT * FROM reviewer")
    reviewer_df = pd.DataFrame(cursor.fetchall(), columns=[column[0] for column in cursor.description])

    # Create Restaurant DataFrame
    cursor.execute("SELECT restaurantID, rating as restaurantRating FROM restaurant")
    restaurant_df = pd.DataFrame(cursor.fetchall(), columns=[column[0] for column in cursor.description])

    # Merge all DataFrames
    review_reviewer_df = review_df.merge(reviewer_df, on='reviewerID', how='inner')
    df = review_reviewer_df.merge(restaurant_df, on='restaurantID', how='inner')
    return df

In [3]:
df = load_data()

In [4]:
fake_df = df[df['flagged'] == 'Y']
fake_df.reset_index(drop=True, inplace=True)

In [5]:
non_fake_df = df[df['flagged'] == 'N']
non_fake_df.reset_index(drop=True, inplace=True)

In [6]:
content_y_legnth = len(fake_df)
content_n_legnth = len(non_fake_df)

In [7]:
ROUND = 10

In [8]:
n_y = content_y_legnth // ROUND
n_n = content_n_legnth // ROUND

In [9]:
content_y_legnth, content_n_legnth

(6206, 20752)

In [10]:
str_y = ''
str_n = ''

In [11]:
for i in range(ROUND):
    if i == ROUND - 1:
        content_y = fake_df.iloc[i*n_y:]
        content_n = non_fake_df.iloc[i*n_n:]
    else:
        content_y = fake_df.iloc[i*n_y:(i+1)*n_y]
        content_n = non_fake_df.iloc[i*n_n:(i+1)*n_n]
    str_y += content_y['reviewContent'].to_string(index=False)
    str_n += content_n['reviewContent'].to_string(index=False)